# Apollo Analytical Model Explorer

Build and simulate the application of Apollo models against an exhaustive dataset.

In [94]:

def project_model_over_trace(data):
    # compile a model
    print("Constructing models:")
    all_models = construct_model_from_flush(data, 'apollo.flush')
    print("Done.")
     
    # go through the trace and evaluate the model
    #
    print("\n\nTrace:")
            
    for row in data['apollo.trace'].itertuples():
        step          = int(row.step)
        region_name   = str(row.region_name)
        policy_index  = int(row.policy_index)
        num_threads   = int(row.num_threads)
        num_elements  = int(row.num_elements)
        time_exec     = float(row.time_exec)
        
        model_policy = int(all_models[region_name].predict([[num_elements]]))
        
        # TODO: calculate the minimum for each region/num_elements
        time_model   = 0.0
        
        # TODO:        
        # eventually have a battery of models being evaluated
        # include the 'perfect' policy/time for contrast
        # include the 'default' policy/time for contrast
        
        time_diff    = float(time_exec - time_model)
        
        # TODO: no more printing, calculate stats and dump summary
        print("      %s %7d  policy %2d took %1.8f seconds" \
              %  (region_name[-17:], num_elements, model_policy, time_model))
            
        if (row.Index > 300):
            print("...")
            break
    #
    return


def plot_apollo_vs_normal(data):
    # Grab the data:
    apollo16 = data['apollo.steps']
    normal16 = data['normal.steps']
    fig, ax = plt.subplots()
    # Refine the data:
    apollo16 = apollo16[apollo16['policy_index']==7].groupby(['policy_index'])
    normal16 = normal16[normal16['num_threads']==16]
    normal16 = normal16[normal16['schedule']=='static'].groupby(['num_threads'])
    apollo_labels = ["apollo." + str(x) for x in apollo16['policy_index'].unique()]
    normal_labels = ["normal." + str(x) for x in normal16['num_threads'].unique()]
    # Plot the refined data:
    apollo16.plot(title="Cleverleaf Step Exec Time(sec) for OMP @ 16 x Static",
                  x='step', y='step_exec_time', ax=ax, linestyle='-')
    normal16.plot(x='step', y='step_exec_time', ax=ax, linestyle=':')
    # Customize plot display:
    ax.legend((apollo_labels + normal_labels), loc='upper right')
    ax.tick_params(axis='y', which='minor', left=True)
    ax.tick_params(axis='x', which='minor', bottom=True)
    plt.grid(True)
    plt.xlabel('Cleverleaf Step')
    plt.ylabel('Time (seconds)')
    plt.show()
    return

#----------

%run './common_routines.ipynb'

data = {}
data['path'] = '/g/g17/wood67/src/apollo/data/intel/001.node.001.rank'
data['apollo.tracefile'] = 'trace.policy.0.csv'
data['apollo.flushfile'] = 'flush.grouped.csv'
data['apollo.stepsfile'] = 'steps.apollo.silent.csv'
data['normal.stepsfile'] = 'steps.normal.silent.csv'
data = load_csv_data(data)

project_model_over_trace(data)
#plot_apollo_vs_normal(data)

print("\nDone.")
return





                \     _ \   _ \  |     |      _ \  
               _ \   |   | |   | |     |     |   | 
              ___ \  ___/  |   | |     |     |   | 
            _/    _\_|    \___/ _____|_____|\___/  
                                                   
           -  -  -  --  --  ---  --= --== ==*# ###>
      
Data source:
	/g/g17/wood67/src/apollo/data/intel/001.node.001.rank

Loading:
       data[apollo.trace]    78.053 MB   trace.policy.0.csv
       data[apollo.flush]     2.507 MB   flush.grouped.csv
       data[apollo.steps]    88.867 kB   steps.apollo.silent.csv
       data[normal.steps]    80.127 kB   steps.normal.silent.csv

Constructing models:
    td.shape = (21196, 6)
grp_td.shape = (8000, 5)
len(region_names) = 71

unique_policies = [1, 14, 11, 8, 3, 9, 10, 15, 4, 5, 2, 16, 17, 18, 19, 12, 6, 0, 13, 7]

Training...
Done.


Trace:
      release(0xe4e3ae)     705  policy 16 took 0.00000000 seconds
      release(0xe47b8e)     704  policy 11 took 0.00000000 seconds
     